In [23]:
import spacy
from spacy import displacy

# Tải mô hình tiếng Anh đã cài đặt
# Sử dụng en_core_web_md vì nó chứa các vector từ và cây cú pháp đầy đủ
nlp = spacy.load("en_core_web_md")
# Câu ví dụ
text = "The quick brown fox jumps over the lazy dog."
# Phân tích câu với pipeline của spaCy
doc = nlp(text)

In [24]:
# Tùy chọn để hiển thị trong trình duyệt
# options = {"compact": True, "color": "blue", "font": "Source Sans Pro"}
# Khởi chạy server tại http://127.0.0.1:5000
# Bạn có thể truy cập địa chỉ này trên trình duyệt để xem cây phụ thuộc
# Nhấn Ctrl+C trong terminal để dừng server
displacy.serve(doc, style="dep")


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


* Từ nào là gốc (ROOT) của câu? jumps
* jumps có những từ phụ thuộc (dependent) nào? Các quan hệ đó là gì? fox (nsubj) over (prep)
* fox là head của những từ nào? The, quick, brown

In [25]:
# Lấy một câu khác để phân tích
text = "The quick brown fox jumps over the lazy dog."
doc = nlp(text)

# In ra thông tin của từng token
print(f"{'TEXT':<12} | {'DEP':<10} | {'HEAD TEXT':<12} | {'HEAD POS':<8} | {'CHILDREN'}")
print("-" * 70)

for token in doc:
    # Trích xuất các thuộc tính
    children = [child.text for child in token.children]
    print(f"{token.text:<12} | {token.dep_:<10} | {token.head.text:<12} | {token.head.pos_:<8} | {children}")


TEXT         | DEP        | HEAD TEXT    | HEAD POS | CHILDREN
----------------------------------------------------------------------
The          | det        | fox          | NOUN     | []
quick        | amod       | fox          | NOUN     | []
brown        | amod       | fox          | NOUN     | []
fox          | nsubj      | jumps        | VERB     | ['The', 'quick', 'brown']
jumps        | ROOT       | jumps        | VERB     | ['fox', 'over', '.']
over         | prep       | jumps        | VERB     | ['dog']
the          | det        | dog          | NOUN     | []
lazy         | amod       | dog          | NOUN     | []
dog          | pobj       | over         | ADP      | ['the', 'lazy']
.            | punct      | jumps        | VERB     | []


In [26]:
# Lấy một câu khác để phân tích
text = "Apple is looking at buying U.K. startup for $1 billion"
doc = nlp(text)

# In ra thông tin của từng token
print(f"{'TEXT':<12} | {'DEP':<10} | {'HEAD TEXT':<12} | {'HEAD POS':<8} | {'CHILDREN'}")
print("-" * 70)

for token in doc:
    # Trích xuất các thuộc tính
    children = [child.text for child in token.children]
    print(f"{token.text:<12} | {token.dep_:<10} | {token.head.text:<12} | {token.head.pos_:<8} | {children}")


TEXT         | DEP        | HEAD TEXT    | HEAD POS | CHILDREN
----------------------------------------------------------------------
Apple        | nsubj      | looking      | VERB     | []
is           | aux        | looking      | VERB     | []
looking      | ROOT       | looking      | VERB     | ['Apple', 'is', 'at']
at           | prep       | looking      | VERB     | ['buying']
buying       | pcomp      | at           | ADP      | ['startup']
U.K.         | compound   | startup      | NOUN     | []
startup      | dobj       | buying       | VERB     | ['U.K.', 'for']
for          | prep       | startup      | NOUN     | ['billion']
$            | quantmod   | billion      | NUM      | []
1            | compound   | billion      | NUM      | []
billion      | pobj       | for          | ADP      | ['$', '1']


In [27]:
text = "The cat chased the mouse and the dog watched them."
doc = nlp(text)
for token in doc:
    # Chỉ tìm các động từ
    if token.pos_ == "VERB":
        verb = token.text
        subject = ""
        obj = ""
        # Tìm chủ ngữ (nsubj) và tân ngữ (dobj) trong các con của động từ
        for child in token.children:
            if child.dep_ == "nsubj":
                subject = child.text
            if child.dep_ == "dobj":
                obj = child.text
        if subject and obj:
            print(f"Found Triplet: ({subject}, {verb}, {obj})")

Found Triplet: (cat, chased, mouse)
Found Triplet: (dog, watched, them)


In [28]:
text = "The big, fluffy white cat is sleeping on the warm mat."
doc = nlp(text)
for token in doc:
    # Chỉ tìm các danh từ
    if token.pos_ == "NOUN":
        adjectives = []
        # Tìm các tính từ bổ nghĩa (amod) trong các con của danh từ
        for child in token.children:
            if child.dep_ == "amod":
                adjectives.append(child.text)
        if adjectives:
            print(f"Danh từ '{token.text}' được bổ nghĩa bởi các tính từ: {adjectives}")

Danh từ 'cat' được bổ nghĩa bởi các tính từ: ['big', 'fluffy', 'white']
Danh từ 'mat' được bổ nghĩa bởi các tính từ: ['warm']


In [29]:
# Bài 1: Tìm động từ chính của câu
def find_main_verb(doc):
    for token in doc:
        # Kiểm tra nếu token là ROOT và là động từ
        if token.dep_ == "ROOT" and token.pos_ == "VERB":
            return token
    
    # Trường hợp dự phòng: Trả về ROOT bất kể POS (ví dụ câu "He is happy" thì happy là ROOT nhưng là ADJ)
    for token in doc:
        if token.dep_ == "ROOT":
            return token
    return None


text = "The quick brown fox jumps over the lazy dog."
doc = nlp(text)
main_verb = find_main_verb(doc)
if main_verb:
    print(text)
    print(f"Động từ chính (ROOT): {main_verb.text} ({main_verb.pos_})")


The quick brown fox jumps over the lazy dog.
Động từ chính (ROOT): jumps (VERB)


In [30]:
# Bài 2: Trích xuất các cụm danh từ (Noun Chunks)
def extract_noun_chunks_manual(doc):
    chunks = []
    for token in doc:
        # Chỉ xét các danh từ (NOUN) hoặc danh từ riêng (PROPN)
        if token.pos_ in ["NOUN", "PROPN"]:
            # Tìm các từ con có quan hệ bổ nghĩa
            modifiers = [child for child in token.children if child.dep_ in ["det", "amod", "compound"]]
            
            # Kết hợp danh từ và các từ bổ nghĩa
            chunk_tokens = modifiers + [token]
            
            # Sắp xếp lại theo thứ tự xuất hiện trong câu
            chunk_tokens.sort(key=lambda t: t.i)
            
            # Tạo chuỗi văn bản từ các token
            chunk_text = " ".join([t.text for t in chunk_tokens])
            chunks.append(chunk_text)
    return chunks

text = "The quick brown fox jumps over the lazy dog."
doc = nlp(text)

print(text)
print("Manual Noun Chunks:", extract_noun_chunks_manual(doc))
print("SpaCy Noun Chunks: ", [chunk.text for chunk in doc.noun_chunks])


The quick brown fox jumps over the lazy dog.
Manual Noun Chunks: ['The quick brown fox', 'the lazy dog']
SpaCy Noun Chunks:  ['The quick brown fox', 'the lazy dog']


In [31]:
# Bài 3: Tìm đường đi ngắn nhất trong cây
def get_path_to_root(token):
    """
    Tìm đường đi từ một token lên đến gốc (ROOT).
    """
    path = [token]
    current = token
    # Duyệt ngược lên head cho đến khi gặp ROOT (token có dep_ là ROOT hoặc head là chính nó)
    while current.dep_ != "ROOT":
        current = current.head
        path.append(current)
    return path


text = "The quick brown fox jumps over the lazy dog."
doc = nlp(text)

target_token = doc[2] # "brown"
path = get_path_to_root(target_token)

print(text)
print(f"Đường đi từ '{target_token.text}' đến ROOT:")
print(" -> ".join([t.text for t in path]))


The quick brown fox jumps over the lazy dog.
Đường đi từ 'brown' đến ROOT:
brown -> fox -> jumps
